In [1]:
%run _standard_imports.ipynb
%run _shared_setup.ipynb


python 3.4.3 |Anaconda 2.2.0 (64-bit)| (default, Mar  6 2015, 12:03:53) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
numpy 1.9.2
scipy 0.15.1
pandas 0.15.2
numexpr 2.3.1
pysam 0.8.3
pysamstats 0.23
petl 1.0.11
petlx 1.0.3
vcf 0.6.7
vcfnp 2.3.0.dev0
h5py 2.4.0
tables 3.1.1


In [9]:
release5_final_files_dir = '/nfs/team112_internal/production/release_build/Pf3K/pilot_5_0'
# chrom_vcf_fn = "%s/SNP_INDEL_Pf3D7_04_v3.combined.filtered.vcf.gz" % (release5_final_files_dir)

output_dir = "/lustre/scratch109/malaria/rp7/data/pf3k/pilot_5_0/20160718_Thomas_5_validation_vcf"
previous_output_dir = "/lustre/scratch109/malaria/rp7/data/pf3k/pilot_5_0/20160524_Thomas_5_validation_vcf_and_CL"

!mkdir -p {output_dir}

In [3]:
BCFTOOLS = '/nfs/team112_internal/rp7/src/github/malariagen/methods-dev/pf3k_techbm/opt_4/bcftools/bcftools'

In [4]:
validation_ids = ['7G8', 'GB4', 'ERS740940', 'ERS740937', 'ERS740936']

In [10]:
# Create Illumina VCF for Thomas
wg_vcf_fn = "%s/SNP_INDEL_WG.combined.filtered.vcf.gz" % (release5_final_files_dir)

validation_samples = ','.join(validation_ids)

# subset_vcf_fn = "%s/SNP_INDEL_WG.combined.filtered.validation.vcf.gz" %  output_dir
# biallelic_vcf_fn = "%s/SNP_INDEL_WG.biallelic.vcf.gz" %  output_dir
subset_vcf_fn = "%s/SNP_INDEL_WG.combined.filtered.validation.vcf.gz" %  previous_output_dir
biallelic_vcf_fn = "%s/SNP_INDEL_WG.biallelic.vcf.gz" %  previous_output_dir
# regenotyped_vcf_fn = "%s/SNP_INDEL_WG.regenotyped.vcf.gz" %  output_dir
new_af_vcf_fn = "%s/SNP_INDEL_WG.new_af.vcf.gz" %  output_dir
nonref_vcf_fn = "%s/SNP_INDEL_WG.nonref.vcf.gz" %  output_dir
minimal_vcf_fn = "%s/SNP_INDEL_WG.minimal.vcf.gz" %  output_dir
thomas_vcf_fn = "%s/SNP_INDEL_WG.for_thomas.vcf.gz" %  output_dir
pass_vcf_fn = "%s/SNP_INDEL_WG.pass.vcf.gz" %  output_dir

# if not os.path.exists(subset_vcf_fn):
#     !{BCFTOOLS} view -Oz -o {subset_vcf_fn} -s {validation_samples} {wg_vcf_fn}
#     !{BCFTOOLS} index --tbi {subset_vcf_fn}

# if not os.path.exists(biallelic_vcf_fn):
#     !{BCFTOOLS} annotate --remove FORMAT/DP,FORMAT/GQ,FORMAT/PGT,FORMAT/PID,FORMAT/PL {subset_vcf_fn} | \
#     {BCFTOOLS} norm -m -any -Oz -o {biallelic_vcf_fn}
#     !{BCFTOOLS} index --tbi {biallelic_vcf_fn}

# if not os.path.exists(regenotyped_vcf_fn):
#     !/nfs/team112/software/htslib/vfp/vfp_tool {biallelic_vcf_fn} /nfs/team112/software/htslib/vfp/just_call.config | \
#     bgzip -c > {regenotyped_vcf_fn}
#     !{BCFTOOLS} index --tbi {regenotyped_vcf_fn}

# This step no longer necessary as not regenotyping, so AD field already correct
# if not os.path.exists(new_af_vcf_fn):
#     !{BCFTOOLS} view -Oz -o {new_af_vcf_fn} {biallelic_vcf_fn}
#     !{BCFTOOLS} index --tbi {new_af_vcf_fn}

if not os.path.exists(nonref_vcf_fn):
    !{BCFTOOLS} view --include 'AC>0 && ALT!="*"' -Oz -o {nonref_vcf_fn} {biallelic_vcf_fn}
    !{BCFTOOLS} index --tbi {nonref_vcf_fn}

if not os.path.exists(minimal_vcf_fn):
    !{BCFTOOLS} annotate --remove INFO/AF,INFO/BaseQRankSum,INFO/ClippingRankSum,INFO/DP,INFO/DS,INFO/END,INFO/FS,\
INFO/GC,INFO/HaplotypeScore,INFO/InbreedingCoeff,INFO/MLEAC,INFO/MLEAF,INFO/MQ,INFO/MQRankSum,INFO/NEGATIVE_TRAIN_SITE,\
INFO/POSITIVE_TRAIN_SITE,INFO/QD,INFO/ReadPosRankSum,INFO/RegionType,INFO/RPA,INFO/RU,INFO/SNPEFF_AMINO_ACID_CHANGE,\
INFO/SNPEFF_CODON_CHANGE,INFO/SNPEFF_EFFECT,INFO/SNPEFF_EXON_ID,INFO/SNPEFF_FUNCTIONAL_CLASS,INFO/SNPEFF_GENE_BIOTYPE,\
INFO/SNPEFF_GENE_NAME,INFO/SNPEFF_IMPACT,INFO/SNPEFF_TRANSCRIPT_ID,INFO/SOR,INFO/STR,INFO/VariantType,INFO/culprit,INFO/set, \
-Oz -o {minimal_vcf_fn} \
{nonref_vcf_fn}
    !{BCFTOOLS} index --tbi {minimal_vcf_fn}

if not os.path.exists(thomas_vcf_fn):
    !{BCFTOOLS} norm --fasta-ref {GENOME_FN} -Oz -o {thomas_vcf_fn} {minimal_vcf_fn}
    !{BCFTOOLS} index --tbi {thomas_vcf_fn}

if not os.path.exists(pass_vcf_fn):
    !{BCFTOOLS} view -f PASS -Oz -o {pass_vcf_fn} {thomas_vcf_fn}
    !{BCFTOOLS} index --tbi {pass_vcf_fn}






Lines   total/split/realigned/skipped:	323786/0/153221/0


In [13]:
# Sanity check what was changed by left aligning and normalising indels
left_align_norm_diffs_dir = "%s/left_align_norm_diffs" %  output_dir
!{BCFTOOLS} isec -p {left_align_norm_diffs_dir} -n-1 -c all {minimal_vcf_fn} {thomas_vcf_fn}

In [15]:
# Create PacBio VCF for Thomas
pacbio_wg_vcf_fn = "/nfs/team112_internal/rp7/data/pf3k/pacbio_1/vcf/pf3k_pacbio_1.vcf.gz"

biallelic_vcf_fn = "%s/pf3k_pacbio_1.biallelic.vcf.gz" %  output_dir
nonref_vcf_fn = "%s/pf3k_pacbio_1.nonref.vcf.gz" %  output_dir
minimal_vcf_fn = "%s/pf3k_pacbio_1.minimal.vcf.gz" %  output_dir
thomas_vcf_fn = "%s/pf3k_pacbio_1.for_thomas.vcf.gz" %  output_dir
pass_vcf_fn = "%s/pf3k_pacbio_1.pass.vcf.gz" %  output_dir

if not os.path.exists(biallelic_vcf_fn):
    !{BCFTOOLS} annotate --remove FORMAT/DP,FORMAT/GQ,FORMAT/PGT,FORMAT/PID,FORMAT/PL {pacbio_wg_vcf_fn} | \
    {BCFTOOLS} norm -m -any -Oz -o {biallelic_vcf_fn}
    !{BCFTOOLS} index --tbi {biallelic_vcf_fn}

if not os.path.exists(nonref_vcf_fn):
    !{BCFTOOLS} view --include 'AC>0 && ALT!="*"' -Oz -o {nonref_vcf_fn} {biallelic_vcf_fn}
    !{BCFTOOLS} index --tbi {nonref_vcf_fn}

if not os.path.exists(minimal_vcf_fn):
    !{BCFTOOLS} annotate --remove INFO/AF,INFO/BaseQRankSum,INFO/ClippingRankSum,INFO/DP,INFO/DS,INFO/END,INFO/FS,\
INFO/GC,INFO/HaplotypeScore,INFO/InbreedingCoeff,INFO/MLEAC,INFO/MLEAF,INFO/MQ,INFO/MQRankSum,INFO/NEGATIVE_TRAIN_SITE,\
INFO/POSITIVE_TRAIN_SITE,INFO/QD,INFO/ReadPosRankSum,INFO/RegionType,INFO/RPA,INFO/RU,INFO/SNPEFF_AMINO_ACID_CHANGE,\
INFO/SNPEFF_CODON_CHANGE,INFO/SNPEFF_EFFECT,INFO/SNPEFF_EXON_ID,INFO/SNPEFF_FUNCTIONAL_CLASS,INFO/SNPEFF_GENE_BIOTYPE,\
INFO/SNPEFF_GENE_NAME,INFO/SNPEFF_IMPACT,INFO/SNPEFF_TRANSCRIPT_ID,INFO/SOR,INFO/STR,INFO/VariantType,INFO/culprit,INFO/set, \
-Oz -o {minimal_vcf_fn} \
{nonref_vcf_fn}
    !{BCFTOOLS} index --tbi {minimal_vcf_fn}

if not os.path.exists(thomas_vcf_fn):
    !{BCFTOOLS} norm --fasta-ref {GENOME_FN} -Oz -o {thomas_vcf_fn} {minimal_vcf_fn}
    !{BCFTOOLS} index --tbi {thomas_vcf_fn}

if not os.path.exists(pass_vcf_fn):
    !{BCFTOOLS} view -f PASS -Oz -o {pass_vcf_fn} {thomas_vcf_fn}
    !{BCFTOOLS} index --tbi {pass_vcf_fn}


Lines   total/split/realigned/skipped:	251945/97387/0/0
Lines   total/split/realigned/skipped:	414363/0/114069/0


In [18]:
# Create PacBio 2 VCF for Thomas
pacbio_wg_vcf_fn = "/nfs/team112_internal/rp7/data/pf3k/pacbio_2/vcf/pf3k_pacbio_2.vcf.gz"

biallelic_vcf_fn = "%s/pf3k_pacbio_2.biallelic.vcf.gz" %  output_dir
nonref_vcf_fn = "%s/pf3k_pacbio_2.nonref.vcf.gz" %  output_dir
minimal_vcf_fn = "%s/pf3k_pacbio_2.minimal.vcf.gz" %  output_dir
thomas_vcf_fn = "%s/pf3k_pacbio_2.for_thomas.vcf.gz" %  output_dir
pass_vcf_fn = "%s/pf3k_pacbio_2.pass.vcf.gz" %  output_dir

if not os.path.exists(biallelic_vcf_fn):
    !{BCFTOOLS} annotate --remove FORMAT/DP,FORMAT/GQ,FORMAT/PGT,FORMAT/PID,FORMAT/PL {pacbio_wg_vcf_fn} | \
    {BCFTOOLS} norm -m -any -Oz -o {biallelic_vcf_fn}
    !{BCFTOOLS} index --tbi {biallelic_vcf_fn}

if not os.path.exists(nonref_vcf_fn):
    !{BCFTOOLS} view --include 'AC>0 && ALT!="*"' -Oz -o {nonref_vcf_fn} {biallelic_vcf_fn}
    !{BCFTOOLS} index --tbi {nonref_vcf_fn}

if not os.path.exists(minimal_vcf_fn):
    !{BCFTOOLS} annotate --remove INFO/AF,INFO/BaseQRankSum,INFO/ClippingRankSum,INFO/DP,INFO/DS,INFO/END,INFO/FS,\
INFO/GC,INFO/HaplotypeScore,INFO/InbreedingCoeff,INFO/MLEAC,INFO/MLEAF,INFO/MQ,INFO/MQRankSum,INFO/NEGATIVE_TRAIN_SITE,\
INFO/POSITIVE_TRAIN_SITE,INFO/QD,INFO/ReadPosRankSum,INFO/RegionType,INFO/RPA,INFO/RU,INFO/SNPEFF_AMINO_ACID_CHANGE,\
INFO/SNPEFF_CODON_CHANGE,INFO/SNPEFF_EFFECT,INFO/SNPEFF_EXON_ID,INFO/SNPEFF_FUNCTIONAL_CLASS,INFO/SNPEFF_GENE_BIOTYPE,\
INFO/SNPEFF_GENE_NAME,INFO/SNPEFF_IMPACT,INFO/SNPEFF_TRANSCRIPT_ID,INFO/SOR,INFO/STR,INFO/VariantType,INFO/culprit,INFO/set, \
-Oz -o {minimal_vcf_fn} \
{nonref_vcf_fn}
    !{BCFTOOLS} index --tbi {minimal_vcf_fn}

if not os.path.exists(thomas_vcf_fn):
    !{BCFTOOLS} norm --fasta-ref {GENOME_FN} -Oz -o {thomas_vcf_fn} {minimal_vcf_fn}
    !{BCFTOOLS} index --tbi {thomas_vcf_fn}

if not os.path.exists(pass_vcf_fn):
    !{BCFTOOLS} view -f PASS -Oz -o {pass_vcf_fn} {thomas_vcf_fn}
    !{BCFTOOLS} index --tbi {pass_vcf_fn}


Lines   total/split/realigned/skipped:	259931/101637/0/0
Lines   total/split/realigned/skipped:	428336/0/118206/0


In [19]:
thomas_vcf_fn

'/lustre/scratch109/malaria/rp7/data/pf3k/pilot_5_0/20160718_Thomas_5_validation_vcf/pf3k_pacbio_2.for_thomas.vcf.gz'

In [16]:
# Create Illumina 16 100bp VCF for Thomas
mrs_1_wg_vcf_fn = "/nfs/team112_internal/rp7/data/pf3k/mrs_1/vcf/pf3k_mrs_1.vcf.gz"

biallelic_vcf_fn = "%s/pf3k_mrs_1.biallelic.vcf.gz" %  output_dir
nonref_vcf_fn = "%s/pf3k_mrs_1.nonref.vcf.gz" %  output_dir
minimal_vcf_fn = "%s/pf3k_mrs_1.minimal.vcf.gz" %  output_dir
thomas_vcf_fn = "%s/pf3k_mrs_1.for_thomas.vcf.gz" %  output_dir
pass_vcf_fn = "%s/pf3k_mrs_1.pass.vcf.gz" %  output_dir

if not os.path.exists(biallelic_vcf_fn):
    !{BCFTOOLS} annotate --remove FORMAT/DP,FORMAT/GQ,FORMAT/PGT,FORMAT/PID,FORMAT/PL {mrs_1_wg_vcf_fn} | \
    {BCFTOOLS} norm -m -any -Oz -o {biallelic_vcf_fn}
    !{BCFTOOLS} index --tbi {biallelic_vcf_fn}

if not os.path.exists(nonref_vcf_fn):
    !{BCFTOOLS} view --include 'AC>0 && ALT!="*"' -Oz -o {nonref_vcf_fn} {biallelic_vcf_fn}
    !{BCFTOOLS} index --tbi {nonref_vcf_fn}

if not os.path.exists(minimal_vcf_fn):
    !{BCFTOOLS} annotate --remove INFO/AF,INFO/BaseQRankSum,INFO/ClippingRankSum,INFO/DP,INFO/DS,INFO/END,INFO/FS,\
INFO/GC,INFO/HaplotypeScore,INFO/InbreedingCoeff,INFO/MLEAC,INFO/MLEAF,INFO/MQ,INFO/MQRankSum,INFO/NEGATIVE_TRAIN_SITE,\
INFO/POSITIVE_TRAIN_SITE,INFO/QD,INFO/ReadPosRankSum,INFO/RegionType,INFO/RPA,INFO/RU,INFO/SNPEFF_AMINO_ACID_CHANGE,\
INFO/SNPEFF_CODON_CHANGE,INFO/SNPEFF_EFFECT,INFO/SNPEFF_EXON_ID,INFO/SNPEFF_FUNCTIONAL_CLASS,INFO/SNPEFF_GENE_BIOTYPE,\
INFO/SNPEFF_GENE_NAME,INFO/SNPEFF_IMPACT,INFO/SNPEFF_TRANSCRIPT_ID,INFO/SOR,INFO/STR,INFO/VariantType,INFO/culprit,INFO/set, \
-Oz -o {minimal_vcf_fn} \
{nonref_vcf_fn}
    !{BCFTOOLS} index --tbi {minimal_vcf_fn}

if not os.path.exists(thomas_vcf_fn):
    !{BCFTOOLS} norm --fasta-ref {GENOME_FN} -Oz -o {thomas_vcf_fn} {minimal_vcf_fn}
    !{BCFTOOLS} index --tbi {thomas_vcf_fn}

if not os.path.exists(pass_vcf_fn):
    !{BCFTOOLS} view -f PASS -Oz -o {pass_vcf_fn} {thomas_vcf_fn}
    !{BCFTOOLS} index --tbi {pass_vcf_fn}


Lines   total/split/realigned/skipped:	574132/177620/0/0
Lines   total/split/realigned/skipped:	737644/0/95757/0


In [14]:
!{BCFTOOLS} isec -p {left_align_norm_diffs_dir} {minimal_vcf_fn} {thomas_vcf_fn}